In [1]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import warnings
warnings.filterwarnings("ignore")



2023-04-29 12:18:39.704878: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/kb/rb5nc0t515n7y9vftjt10s9h0000gn/T/ipykernel_58148/2554462468.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
#  Import and read the charity_data.csv.
final_application_df = pd.read_csv("final_application.csv")


In [3]:
final_application_df

,Unnamed: 0,STATUS,ASK_AMT,APPLICATION_TYPE_OTHER,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,IS_SUCCESSFUL
0,0,1,5000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,1,1,108590,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
2,2,1,5000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,3,1,6692,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,1
4,4,1,142590,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,34294,1,5000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
34295,34295,1,5000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
34296,34296,1,5000,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
34297,34297,1,5000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1


In [6]:
# Split our preprocessed data into our features and target arrays
# Convert input to numpy array
np_application_df = final_application_df.to_numpy()

X_data = np_application_df[:,0:44]
y_data= np_application_df[:,44]

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split( X_data, y_data, random_state=1)
  

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Allow kerastuner to decide which model optimizer to us
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adagrad', "sgd"])
    
    # epochs = hp.Choice('num_epochs', [10, 20, 30, 40, 50, 60])
    
    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    return nn_model

In [9]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=200,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [10]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test))

INFO:tensorflow:Oracle triggered exit


In [11]:
# Get top 10model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(5)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 16, 'num_layers': 4, 'units_0': 16, 'units_1': 6, 'units_2': 16, 'units_3': 6, 'units_4': 26, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 26, 'num_layers': 3, 'units_0': 26, 'units_1': 26, 'units_2': 11, 'units_3': 16, 'units_4': 26, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 1, 'num_layers': 4, 'units_0': 21, 'units_1': 11, 'units_2': 21, 'units_3': 11, 'units_4': 1, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0036'}
{'activation': 'tanh', 'first_units': 21, 'num_layers': 4, 'units_0': 26, 'units_1': 26, 'units_2': 11, 'units_3': 21, 'units_4': 11, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0038'}
{'activation': 'relu', 'first_units': 1, 'num_layers': 4, 'units_0': 16, 'units_1

In [13]:
# Get first best model hyperparameters
first_hyper = tuner.get_best_hyperparameters(1)[0]
first_hyper.values

{'activation': 'tanh',
 'first_units': 16,
 'num_layers': 4,
 'units_0': 16,
 'units_1': 6,
 'units_2': 16,
 'units_3': 6,
 'units_4': 26,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0,
 'optimizer': 'adam'}